***
## Создание класса формы

В приложении *birthday* создайте файл *forms.py*; в Django-проектах файл, в котором хранят код, управляющий формами, традиционно называют именно так.

Для создания формы создадим собственный класс, унаследованный от встроенного класса **Form**. В приложении **birthday** будет несколько форм, и для каждой потребуется создать отдельный класс. 

Настроим первую форму в проекте; в первом приближении она должна быть такой:

![alt text](https://pictures.s3.yandex.net/resources/119_1682595621.png)

В файле *forms.py* опишите класс формы `BirthdayForm`, унаследованный от базового класса `django.forms.Form`.

Каждое из полей, которое должно быть у веб-формы, описывается как атрибут класса `BirthdayForm`. Тип каждого поля указывается с помощью специальных классов из модуля `django.forms`; названия этих классов выглядят как `ТипПоляField`:

* `IntegerField` для целочисленных полей,

* `CharField` для текстовых полей,

* `DateField` для полей с датой.

Полный список доступных типов полей можно посмотреть [в документации](https://docs.djangoproject.com/en/3.2/ref/forms/fields/#built-in-field-classes). 

```py
# birthday/forms.py
from django import forms


class BirthdayForm(forms.Form):
    first_name = forms.CharField(max_length=20)
    last_name = forms.CharField(required=False)
    birthday = forms.DateField() 
```

Класс формы готов, теперь нужно настроить приложение так, чтобы при обращении к адресу *birthday/* пользователю возвращалась страница с веб-формой, созданной на основе класса `BirthdayForm`.

Маршруты в приложении *birthday* уже настроены; при обращении к адресу *birthday/* вызывается view-функция `birthday()`, которая вызывает шаблон *birthday.html*. 

***
## Шаблон для отображения формы

Форма передаётся в шаблон в словаре контекста под ключом `form` (это традиционное название, хотя назвать этот ключ можно как угодно). Переменная `form` содержит HTML-код всех полей формы, но без тега `<form>` и без кнопки отправки формы; эти элементы кода нужно добавить в шаблон вручную. 

```html
<!-- templates/birthday/birthday.html -->
{% extends "base.html" %}

{% block content %}
  <form>
    {{ form }}
    <input type="submit" value="Submit">
  </form>
{% endblock %} 
```

***
## Работа с формой во view-функции

При запросе к адресу *birthday/* вызывается view-функция `birthday()`, в которой должен быть создан экземпляр класса формы `BirthdayForm`. Этот экземпляр нужно передать в HTML-шаблон через словарь `context`.

Сделаем же это! Дополните файл *birthday/views.py* согласно коду ниже:

```py
# birthday/views.py
from django.shortcuts import render

# Импортируем класс BirthdayForm, чтобы создать экземпляр формы.
from .forms import BirthdayForm


def birthday(request):
    # Создаём экземпляр класса формы.
    form = BirthdayForm()
    # Добавляем его в словарь контекста под ключом form:
    context = {'form': form}
    # Указываем нужный шаблон и передаём в него словарь контекста.
    return render(request, 'birthday/birthday.html', context) 
```

Откройте исходный код страницы (CTRL+U в Windows и Linux, Cmd+Option+U в macOS). В коде страницы видно, что в теге `<form>` описаны все поля формы; они обрамлены тегами `<tr>`, `<th>` и `<td>` — по умолчанию класс Form оформляет веб-форму в виде HTML-таблицы.

```html
<form>
  <tr>
    <th><label for="id_first_name">First name:</label></th>
    <td>
      <input type="text" name="first_name" maxlength="20" required id="id_first_name">
    </td>
  </tr>

  <tr>
    <th><label for="id_last_name">Last name:</label></th>
    <td>
      <input type="text" name="last_name" id="id_last_name">
    </td>
  </tr>

  <tr>
    <th><label for="id_birthday">Birthday:</label></th>
    <td>
      <input type="text" name="birthday" required id="id_birthday">
    </td>
  </tr>
  <input type="submit" value="Submit">
</form> 
```

Поля отображаются в том порядке, в котором они были определены в классе формы. К каждому полю был добавлен тег `<label>` с названием поля. К обязательным полям добавлен атрибут `required`: браузер не позволит отправить форму, если поле с этим атрибутом не заполнено.

***
## Варианты HTML-обрамления формы

Форма, выведенная на страницу *birthday/*, оформлена в виде строк и ячеек HTML-таблицы.

**Таблица** включает в себя строки; в теги строк вложены теги ячеек; в ячейках таблицы описано содержимое

`<tr></tr>` — table row, строка таблицы,

`<td></td>` — table data, ячейка таблицы.

`<th></th>` — table header, тоже ячейка таблицы, особая её разновидность: ячейка для заголовка таблицы.

Подробное описание HTML-таблиц [есть в документации](https://developer.mozilla.org/ru/docs/Learn_web_development/Core/Structuring_content/HTML_table_basics).

Все элементы таблицы в HTML должны быть обёрнуты в тег `<table></table>`, но класс Form этот тег не генерирует, его нужно добавить в шаблон вручную.

Добавьте в код теги `<table>` и `</table>`:

```html
<form>
  <table>
    {{ form }}
  </table>
  <input type="submit" value="Submit">
</form>
```

![alt text](https://pictures.s3.yandex.net/resources/123_1682595726.png)

Для отображения формы есть и другие варианты; указать нужный способ оформления можно с помощью специальных методов. Например, поля можно вывести в виде маркированного списка `<ul>`: для этого в шаблоне указывается метод `form.as_ul`, а вместо тега `<table>` ставится тег `<ul>`.

```html
<form>
  <ul>
    {{ form.as_ul }}
  </ul>
  <input type="submit" value="Submit">
</form> 
```

Форма будет выглядеть так:

![alt text](https://pictures.s3.yandex.net/resources/124_1682595738.png)

Можно применить метод `form.as_p`, тогда каждое поле будет обёрнуто в тег абзаца `<p>`. 

```html
<form>
  {{ form.as_p }}
  <input type="submit" value="Submit">
</form> 
```

И выглядеть форма будет так:

![alt text](https://pictures.s3.yandex.net/resources/247_1682626088.png)


***
## Подписи и подсказки к полям формы

По умолчанию заголовки полей HTML-формы генерируются из имён свойств класса формы: имя свойства `first_name` превратилось в лейбл *First name*, а `last_name` — в *Last name*. 

Названия полей лучше бы выводить по-русски; для этого нужно немного дополнить класс формы `BirthdayForm`. Заодно добавим в форму сообщение о том, что поле с фамилией необязательно для заполнения — такие подсказки тоже добавляются в описание поля в классе формы.

Кастомные названия полей устанавливаются в аргументе `label`, а подсказка для поля — в аргументе `help_text` нужного поля.
 
Допишите нужные атрибуты в класс `BirthdayForm`.

```py
# birthday/forms.py
...
class BirthdayForm(forms.Form):
    first_name = forms.CharField(label='Имя', max_length=20)
    last_name = forms.CharField(
        label='Фамилия', required=False, help_text='Необязательное поле'
    )
    birthday = forms.DateField(label='Дата рождения') 
```

***
## Выбор даты

Имя и фамилию пользователь, скорее всего, напишет правильно, но вот с датой могут возникнуть проблемы: если пользователь ошибётся с форматом — бэкенд не сможет преобразовать полученные данные в формат даты — и, соответственно, не сможет провести необходимые вычисления.

Сейчас никто не мешает пользователю ввести дату рождения в любом виде; что в это поле ни напиши — всё будет отправлено.

Необходимо ввести ограничение на формат вводимой строки: в поле для даты в соответствии с настройками проекта ожидается формат `ГГГГ-ММ-ДД` (например, 1799-06-06). 

При работе с классом **Form** можно указать, какой интерфейс будет предоставлен пользователю для ввода данных. Например, HTML-тег `<input>` может предоставлять пользователю несколько разных интерфейсов (в зависимости от атрибута `type`): поле ввода текста, поле для чисел, чекбокс… 

В описании класса можно указать, какой именно интерфейс нужен для того или иного поля — это делается с помощью **виджетов**.

По умолчанию к каждому типу поля уже подключён определённый виджет, но его можно заменить. Список всех встроенных виджетов можно посмотреть [в документации](https://docs.djangoproject.com/en/3.2/ref/forms/widgets/#built-in-widgets).

Для поля с датой подойдёт виджет, позволяющий пользователю выбрать нужную дату на календаре. Этот виджет не позволит ввести дату в неправильном формате или указать несуществующую дату. При этом у пользователя остаётся возможность ввести дату текстом — и она сразу будет проверена на валидность.

Виджеты указывают в аргументе `widget`; добавьте виджет календаря к полю `birthday` в файле *forms.py*.

```py
# birthday/forms.py
...
class BirthdayForm(forms.Form):
    first_name = forms.CharField(label='Имя', max_length=20)
    last_name = forms.CharField(
        label='Фамилия', required=False, help_text='Необязательное поле'
    )
    birthday = forms.DateField(
        label='Дата рождения',
        # Указываем, что виджет для ввода даты должен быть с типом date.
        widget=forms.DateInput(attrs={'type': 'date'})
    )
```

В аргументе `attrs` задан атрибут `type="date"` для тега `<input>`. Если HTML-форму верстают вручную, этот атрибут описывают прямо в теге:` <input type="date">`.

Описание атрибута `type="date"` тега `<input>` можно посмотреть [в справочнике по HTML](https://developer.mozilla.org/ru/docs/Web/HTML/Reference/Elements/input/date).

Подробнее про виджеты можно прочитать [в документации](https://docs.djangoproject.com/en/3.2/ref/forms/widgets/).